# 修了課題DEMO④　CIFAR10



##はじめに
この修了課題では、Pytorchを使って「CIFAR10」という画像データセットの分類を行っていきます。  
今回使用するネットワーク構成は
    
    conv2d - relu - MaxPool2d -
    conv2d - relu - MaxPool2d -
    flatten -
    Linear - relu -
    Linear - relu - Linear
上記のようなCNNとなっています。  
ここで、viewとはサイズを１度調整するコマンドのことで、各要素の値そのものに変化はありません。  
また、Linearは線形結合を表しています。


##作成までの流れ
大まかな流れとして
1. データのダウンロードと正規化  
   torchvisionというライブラリを使用して、CIFAR10の訓練用のデータ、テスト用のデータをダウンロードします。  
   また、ダウンロードした画像に対して正規化を行います。

2. モデルの構築  
   学習を行うモデルの各層の役割を理解して、構築します。

3. 損失関数などの設定  
   学習を行うのに必要な損失関数などの設定を行います。

4. 学習と結果  
   訓練データで学習を行い、どのくらいの精度があるのかを、テスト用データを使って確認します。

##必要なライブラリーのインポートとGoogleDriveへの接続

In [ ]:
#GoogleDriveへの接続を行う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#必要なライブラリーのインポート
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#ダウンロードに必要なライブラリーのインポート
import pickle
from PIL import Image
import os

#1.データのダウンロードと正規化

## データのダウンロード

今回使用するデータを「wget」コマンドでダウンロードします。  
ダウンロードしたデータを直列に配置し、画像に変換する処理を行っていますが、  
今回はモデルの作成と学習が目的ですので、気にせず実行してください。

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3' -O train.pickle

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN' -O val.pickle

下のセルで画像ファイルとして保存する処理を行っています。  
実行に平均1分程度かかりますが、データのダウンロードが完了します。

In [ ]:
# バイナリファイルを読み込んでから、画像データに変換処理を行う。
def parse_pickle(rawdata, dataset_name):
    for i in range(10):
        dir = dataset_name + "/" + f"{i:02d}"
        if not os.path.exists(dir):
            os.makedirs(dir)
    m = len(rawdata["data"])
    for i in range(m):
        filename = f'{i}.png'
        label = rawdata["label"][i]
        data = rawdata["data"][i]
        data = data.reshape(3, 32, 32)
        data = np.swapaxes(data, 0, 2)
        data = np.swapaxes(data, 0, 1)
        with Image.fromarray(data) as img:
            img.save(f"{dataset_name}/{label:02d}/{filename}")

train = {'label':[], 'data':[]}
with open('train.pickle', "rb") as fp:
  train = pickle.load(fp, encoding="latin-1")
parse_pickle(train, "train")

with open('val.pickle', "rb") as fp:
  val = pickle.load(fp, encoding="latin-1")
parse_pickle(val, "val")

##データ拡張とデータセット作成
画像認識において「データ拡張」は汎化性能を上げる重要な役割を果たします。  
torchvision.transforms という PyTorch のサブライブラリを用いてデータ拡張を行います。  
また、transforms.Composeの中に記入することで、記入したtransformのコマンドを一度に実行することができます。

今回は一般的に必要な  

*  画像のテンソル化 ( transforms.ToTensor() )  
*  RGBの平均と標準偏差をそれぞれ0.5に設定する正規化 ( transforms.Nomalize())

を行いました。

Pytorchが提供しているその他のtransformのURLを記載しておくので、精度が上がる手法をそれぞれ試してみてください。  
データ拡張:https://pytorch.org/vision/stable/transforms.html

In [ ]:
# データ拡張の設定
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                #もっと増やしてみてもいいかもしれません
                                ])

次にバッチサイズとデータをまとめる処理になります。

In [ ]:
# バッチサイズの設定
batch_size = 25

# データローダーの設定
trainset = torchvision.datasets.ImageFolder(root='train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='val', transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

データの量を確認します。

In [ ]:
print('学習データ:', len(trainset))
print('検証データ:', len(valset))

##データの確認
ここで一度、データがどのようなものなのか確認してみましょう。

# 6.転移学習

5章で作成したVGG11は、重みを１から作成して学習を始めました。  
発展的なモデルはモデルの容量が大きくなった分、１回の学習時間が非常に長くなってしまうという問題点があります。  
そのため、長期間学習を行わないと適した精度まで上がらないため、あまり実用的ではありません。  
そこで、事前にある程度学習が行われている事前学習済みモデルを利用する「転移学習」という手法を用います。

PyTorchの公式サイトに掲載されているように、転移学習を実装したモデルを使ってみましょう。

PyTorchの公式サイト：https://pytorch.org/vision/stable/models.html

In [ ]:
# pytorch のライブラリーを利用して、事前学習の重みをロード済みのモデルインスタンスを作成する。
# なお、pretrained=True とすると事前学習モデルとなり、Falseとするとモデルのみが作成される。
net = torchvision.models.convnext_base(pretrained=True)
net

In [ ]:
# 分類器部分を cifar10 用に付け替える。
net.classifier[2] = nn.Linear(1024 ,out_features=10)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

# 学習エポックの設定
epoch_num = 50

# 学習ループの設定
for epoch in tqdm(range(epoch_num)):  # エポックの進行度を表示するためにtqdmを使用

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # leave=Falseで内部のプログレスバーが完了後に消えるように設定
        inputs, labels = data
        optimizer.zero_grad()

        # テンソルをGPUに移動
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 結果表示
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')


##結果のモデルを保存する


In [ ]:
vgg_pre_weight_path = './vgg_pre_weight_path.pth'
torch.save(net.state_dict(), vgg_pre_weight_path)

##結果を検証用データで確認する

In [ ]:
net.load_state_dict(torch.load(vgg_pre_weight_path))

In [ ]:
correct = 0
total = 0
# 勾配を記憶せず（学習せずに）に計算を行う
with torch.no_grad():
    for data in valloader:
        images, labels = data

        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('正解率 : %d %%' % (100 * correct / total))

## （参考）提出用データの作成の仕方

上記のモデルでテスト用データを予測して提出用ファイルを出力するまでを
掲載してみました。

実行してみましょう。

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-T-luRcFf14qV_rR66B3groh8imA-8lo' -O test_data.pickle

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

with open('test_data.pickle', "rb") as fp:
  test = pickle.load(fp, encoding="latin-1")

for i in range(len(test['data'])):
  data = test["data"][i]
  data = data.reshape(3, 32, 32)
  data = np.swapaxes(data, 0, 2)
  data = np.swapaxes(data, 0, 1)
  img = transform(data)
  img = torch.unsqueeze(img, 0)
  if i==0:
    images=img
  else:
    images = torch.cat([images, img])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.eval()
images = images.to(device)
with torch.no_grad():
    outputs = net(images)
    _, predictions = torch.max(outputs, 1)
print(predictions)

In [ ]:
# pandasのDataFrame形式に変換し、CSV出力する
import pandas as pd
y_pred = pd.DataFrame(predictions.cpu(), columns=['number'])
y_pred.to_csv('y_pred.csv')
y_pred

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-UHqW8wgH46J-ltEdUfOX-DounUbZMAI' -O test_label.pickle
with open('test_label.pickle', "rb") as fp:
  test_label = pickle.load(fp, encoding="latin-1")

In [ ]:
labels = torch.tensor(test_label['label'])
correct = (predictions.cpu() == labels).sum().item()
assert len(predictions) == len(labels)
print( f"正解率 : {100 * correct // len(labels)} %" )